In [46]:
import numpy as np
import sympy as sp
# sp.init_printing()

## Sympy

In [47]:
def curl_sym(Fx, Fy, Fz, x, y, z):
    curl_x = sp.diff(Fz, y) - sp.diff(Fy, z)
    curl_y = sp.diff(Fx, z) - sp.diff(Fz, x)
    curl_z = sp.diff(Fy, x) - sp.diff(Fx, y)
    return curl_x, curl_y, curl_z

In [48]:
x_sym, y_sym, z_sym = sp.symbols('x y z')

# Define 3D velocity field
u_sym = 3*y_sym*z_sym
v_sym = x_sym*z_sym
w_sym = 2*x_sym*y_sym

omega_sym = curl_sym(u_sym, v_sym, w_sym, x_sym, y_sym, z_sym)

print(f"Vel: u = {u_sym}, v = {v_sym}, w = {w_sym}")
print(f"Curl: omega = {omega_sym}")

Vel: u = 3*y*z, v = x*z, w = 2*x*y
Curl: omega = (x, y, -2*z)


## Numpy

In [49]:
def curl_numerical(u, v, w, dx):
    """
    Returns the numerical curl of a vector field.
    """
    def _dim(u):
        return len(np.shape(u))
    if _dim(u) != _dim(v):
        raise ValueError('u and v must have the same dimensionality')
    if _dim(u) == 2:
        du_dy, du_dx = np.gradient(u, dx, edge_order=2)
        dv_dy, dv_dx = np.gradient(v, dx, edge_order=2)
        curl_z = dv_dx - du_dy
        return curl_z
    elif _dim(u) == 3:
        du_dy, du_dx, du_dz = np.gradient(u, dx, edge_order=2)
        dv_dy, dv_dx, dv_dz = np.gradient(v, dx, edge_order=2)
        dw_dy, dw_dx, dw_dz = np.gradient(w, dx, edge_order=2)
        curl_x = dw_dy - dv_dz
        curl_y = du_dz - dw_dx
        curl_z = dv_dx - du_dy
        return curl_x, curl_y, curl_z
    else:
        raise ValueError('Invalid dimensionality')

In [50]:
# Define 3D velocity field
_x = np.linspace(0, 1, 5)
dx = _x[1] - _x[0]
x, y, z = np.meshgrid(_x, _x, _x)
u_func = sp.lambdify((x_sym, y_sym, z_sym), u_sym)
v_func = sp.lambdify((x_sym, y_sym, z_sym), v_sym)
w_func = sp.lambdify((x_sym, y_sym, z_sym), w_sym)
u = u_func(x, y, z)
v = v_func(x, y, z)
w = w_func(x, y, z)
# w = np.zeros_like(u)
vmag = np.sqrt(u**2 + v**2 + w**2)

In [51]:
omega_x, omega_y, omega_z = curl_numerical(u, v, w, dx)

omega_x_exact_func = sp.lambdify((x_sym, y_sym, z_sym), sp.diff(w_sym, y_sym) - sp.diff(v_sym, z_sym))
omega_y_exact_func = sp.lambdify((x_sym, y_sym, z_sym), sp.diff(u_sym, z_sym) - sp.diff(w_sym, x_sym))
omega_z_exact_func = sp.lambdify((x_sym, y_sym, z_sym), sp.diff(v_sym, x_sym) - sp.diff(u_sym, y_sym))
omega_x_exact = omega_x_exact_func(x, y, z)
omega_y_exact = omega_y_exact_func(x, y, z)
omega_z_exact = omega_z_exact_func(x, y, z)

err_x = np.abs(omega_x - omega_x_exact).max()
err_y = np.abs(omega_y - omega_y_exact).max()
err_z = np.abs(omega_z - omega_z_exact).max()

In [52]:
err_x, err_y, err_z

(0.0, 0.0, 0.0)